In [73]:
import numpy as np
from random import randint, random
from copy import deepcopy
from tqdm.auto import tqdm


In [74]:
#Main optimization function: Simulated Annealing
def simulated_annealing_knapsack(VALUES, WEIGHTS, CONSTRAINTS, *,
                                 max_steps=2000,
                                 init_temp=100.0,
                                 cooling_rate=0.995,
                                 tweak_strength=0.3,
                                 penalty_factor=10.0,
                                 verbose=True):
   

    NUM_ITEMS = len(VALUES)
    NUM_KNAPSACKS, NUM_DIMENSIONS = CONSTRAINTS.shape

   #Generate a random solution
    def random_solution():
        #Each item assigned randomly to one knapsack or -1 (not taken)
        return [randint(-1, NUM_KNAPSACKS - 1) for _ in range(NUM_ITEMS)]

    #Evaluate a solution
    def cost(solution):
        #Negative total value + penalties for capacity violations
        penalty = 0
        total_value = 0
        for k in range(NUM_KNAPSACKS):
            used = np.zeros(NUM_DIMENSIONS)
            for i, s in enumerate(solution):
                if s == k:
                    used += WEIGHTS[i]
                    total_value += VALUES[i]
                    #Add penalties for violated constraints
            for d in range(NUM_DIMENSIONS):
                if used[d] > CONSTRAINTS[k, d]:
                    penalty += penalty_factor * (used[d] - CONSTRAINTS[k, d]) / (CONSTRAINTS[k, d] + 1e-9)
        return -total_value + penalty
    
    #Create a modified solution (small perturbation)
    def tweak(solution, strength=tweak_strength):
        new_solution = deepcopy(solution)
        n_changes = max(1, int(len(solution) * strength))
        for _ in range(n_changes):
            i = randint(0, NUM_ITEMS - 1)
            new_solution[i] = randint(-1, NUM_KNAPSACKS - 1)
        return new_solution

   #Simulated Annealing loop
    current_solution = random_solution()
    current_cost = cost(current_solution)
    best_solution, best_cost = deepcopy(current_solution), current_cost
    temp = init_temp

    for step in tqdm(range(max_steps), disable=not verbose):
        #Gradually decrease tweak intensity
        new_solution = tweak(current_solution, strength=max(0.05, tweak_strength * (1 - step / max_steps)))
        new_cost = cost(new_solution)
        diff = new_cost - current_cost
 
        #Accept better solutions or occasionally worse ones (exploration)
        if new_cost < current_cost or random() < np.exp(-diff / temp):
            current_solution, current_cost = new_solution, new_cost
            if current_cost < best_cost:
                best_solution, best_cost = deepcopy(current_solution), current_cost

        #Temperature decay
        temp *= cooling_rate

    return best_solution, best_cost


In [75]:
#Evaluate and print a final solution
def evaluate(solution, VALUES, WEIGHTS, CONSTRAINTS):
    NUM_KNAPSACKS, NUM_DIMENSIONS = CONSTRAINTS.shape
    total_value = 0

    for k in range(NUM_KNAPSACKS):
        used = np.zeros(NUM_DIMENSIONS)
        value = 0
        items = []
        for i, s in enumerate(solution):
            if s == k:
                used += WEIGHTS[i]
                value += VALUES[i]
                items.append(i)
        total_value += value
        print(f"Knapsack {k}: items={items[:10]}{'...' if len(items) > 10 else ''}, "
              f"used={used.round(1)}, capacity={CONSTRAINTS[k]}, value={value}")
    print("Items not taken:", [i for i, s in enumerate(solution) if s == -1][:20])
    print(f"Total value: {total_value}")


In [85]:
#Problem 0
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

VALUES = rng.integers(10, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(5, 50, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

solution, cost_val = simulated_annealing_knapsack(
    VALUES, WEIGHTS, CONSTRAINTS,
    max_steps=1000,
    init_temp=100,
    cooling_rate=0.995,
    verbose=True
)

print("\n--Problem 0 Results--")
print("Best cost:", cost_val)
evaluate(solution, VALUES, WEIGHTS, CONSTRAINTS)


  0%|          | 0/1000 [00:00<?, ?it/s]


--Problem 0 Results--
Best cost: -453.41904761965617
Knapsack 0: items=[0, 1, 2, 3, 4, 5, 9], used=[229. 207.], capacity=[150  75], value=367
Knapsack 1: items=[6, 8], used=[36. 88.], capacity=[ 30 184], value=45
Knapsack 2: items=[7], used=[40. 33.], capacity=[295  21], value=72
Items not taken: []
Total value: 484


**TEST PROBLEMS**

In [77]:
#Problem 1
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

solution, cost_val = simulated_annealing_knapsack(
    VALUES, WEIGHTS, CONSTRAINTS,
    max_steps=2000,
    verbose=True
)

print("\n--Problem 1 Results--")
print("Best cost:", cost_val)
evaluate(solution, VALUES, WEIGHTS, CONSTRAINTS)


  0%|          | 0/2000 [00:00<?, ?it/s]


--Problem 1 Results--
Best cost: -1065
Knapsack 0: items=[2, 5, 7, 8, 10, 11, 13, 15, 19], used=[575. 460.], capacity=[614 496], value=587
Knapsack 1: items=[0, 1, 3, 6, 12, 14, 18], used=[244. 455.], capacity=[244 644], value=363
Knapsack 2: items=[4, 9, 16, 17], used=[166. 157.], capacity=[273 216], value=115
Items not taken: []
Total value: 1065


In [86]:
#Problem 2
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

solution, cost_val = simulated_annealing_knapsack(
    VALUES, WEIGHTS, CONSTRAINTS,
    max_steps=3000,
    cooling_rate=0.998
)
print("\n--Problem 2 Results--")
print("Best cost:", cost_val)
evaluate(solution, VALUES, WEIGHTS, CONSTRAINTS)


  0%|          | 0/3000 [00:00<?, ?it/s]


--Problem 2 Results--
Best cost: -52527.52363902915
Knapsack 0: items=[4, 37, 43, 46, 50, 64, 89, 96], used=[5399. 4002. 3069. 2880. 5995. 3939. 3869. 2442. 5239. 2295.], capacity=[5837 5754 9351 3205 4450 3447 3952 9256 9525 2357], value=4299
Knapsack 1: items=[14, 42, 44, 48, 49, 58, 75, 97], used=[4476. 3667. 3503. 4398. 3422. 3985. 4738. 2842. 5211. 5544.], capacity=[7056 3862 3540 4336 3555 5921 9964 6691 7785 5946], value=4839
Knapsack 2: items=[15, 17, 23, 33, 56, 66, 70, 85, 87], used=[1857. 3932. 4063. 5204. 4598. 4647. 4828. 4784. 3289. 4499.], capacity=[2111 2672 4150 3949 8724 8748 8832 7100 6449 7193], value=5652
Knapsack 3: items=[22, 45, 57, 62, 65, 74, 84, 86, 91, 92]..., used=[5116. 4703. 7514. 4578. 6409. 4359. 6102. 2756. 5799. 5921.], capacity=[5254 7361 8355 8103 6910 2464 7799 4932 7949 6316], value=5441
Knapsack 4: items=[0, 7, 12, 13, 20, 25, 32, 34, 38, 51]..., used=[7749. 7341. 7330. 7564. 9059. 8201. 8913. 8543. 5775. 7521.], capacity=[9049 4707 9451 8755 46

In [87]:
#Problem 3
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))

solution, cost_val = simulated_annealing_knapsack(
    VALUES, WEIGHTS, CONSTRAINTS,
    max_steps=10000,
    init_temp=500,
    cooling_rate=0.999,
    tweak_strength=0.06,
    verbose=True  
)
print("\n--Problem 3 Results--")
print("Best cost:", cost_val)
evaluate(solution, VALUES, WEIGHTS, CONSTRAINTS)


  0%|          | 0/10000 [00:00<?, ?it/s]


--Problem 3 Results--
Best cost: -2482364.553397065
Knapsack 0: items=[535, 659, 776, 915, 984, 1015, 1234, 1265, 1271, 1314]..., used=[22553. 24514. 24923. 25098. 23192. 22963. 21030. 28608. 24200. 23422.
 23802. 18396. 19321. 24180. 21459. 25780. 25985. 21317. 23416. 20819.
 23330. 24599. 19951. 21465. 25806. 22633. 22679. 24941. 22341. 24448.
 24518. 21244. 27597. 19587. 20970. 22342. 21673. 23487. 21892. 19991.
 22125. 19347. 21245. 25040. 21338. 22592. 26938. 24788. 25118. 24900.
 20822. 24068. 22248. 20564. 21683. 23246. 19504. 25184. 21599. 20731.
 21557. 22048. 21791. 24808. 22264. 25391. 20792. 21354. 23038. 23648.
 21349. 25028. 23318. 24413. 22065. 25498. 21230. 23201. 23208. 22647.
 22022. 20992. 25626. 21134. 22667. 24648. 20635. 24519. 22980. 27479.
 23346. 22469. 23136. 21708. 24328. 19647. 21464. 22876. 25260. 22925.], capacity=[79834 52005 17527 30007 59295 98921 12881 36174 63958 90566 25740 12251
 27746 78136 90704 39249 23072 69010 17326 29277 42886 48016 74931 934